In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Chris dataset
df = pd.read_csv('/Users/renatoboemer/code/boemer00/Netflix/raw_data/merged_movies_by_index.csv')

In [4]:
df.shape  # 80% = 8309 |  20% = 2077  

(10386, 34)

## Data Split

In [5]:
df = df[:8309]
df_test = df[8309:]

In [6]:
df.shape

(8309, 34)

In [7]:
df.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test_DONT_TOUCH.csv', index=False)

## Data Wrangling

In [9]:
# questions: index_match?
df = data_wrangling(df)

TypeError: argument of type 'float' is not iterable

In [ ]:
df.shape

## Data Exploration

## Pipeline

## MLFlow

In [ ]:
14-JUL-1789